In [1]:
import warnings
import pandas as pd
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
nlp = spacy.load('es')
nlp = spacy.load('es_core_news_sm')#modelo de spacy
spacy_stopwords = spacy.lang.es.stop_words.STOP_WORDS
from tqdm import tqdm, trange
import time
import matplotlib.pyplot as plt
from itertools import cycle, islice
from sklearn.pipeline import make_pipeline
import logging
from optparse import OptionParser
from TermTagger import term_tagger_git as tt
import re

In [2]:
from TermTagger import term_tagger_git as tt

In [3]:
import os, sys
import seaborn as sns
import PyPDF2
from nltk import Tree
from spacy import displacy
from spacy.pipeline import DependencyParser

## Para el doc pdf

In [62]:
pdfFileObj = open('Plan-de-accion-conjunto-2019.pdf', 'rb')
#pdfFileObj = open('PNCVHM_2009-2015.pdf', 'rb')
#pdfFileObj = open('MoDecoFinalPDF.pdf', 'rb')

In [63]:
## LEER PDF DE LEY Y CONVERTIR A TEXTO PLANO
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
Pages=pdfReader.numPages## LEER PDF Y CONVERTIR A TEXTO PLANO
#Almacenar el contenido por paginas
Caps=[]
for i in range(0,Pages):
    pageObj = pdfReader.getPage(i)
    Cap=pageObj.extractText()
    Caps.append(Cap)

In [64]:
Caps

[' \n1\n \n \n \nPLAN DE ACCIÓN CONJUNTO PARA PREVENIR LA VIOLENCIA CONTRA LAS \nMUJERES, ASÍ COMO BRINDAR PROTECCIÓN Y ATENCIÓN A LAS \nVÍCTIMAS DE VIOLENCIA, CON ÉNFASIS EN LOS CASOS DE ALTO RIESGO \nPARA EL AÑO 2019\n \n \n \nI.\n \nANTECEDENTES\n \n \nAnte los graves hechos de violencia contra las mujeres que ocurren en el país, el 2 \nde junio de 2018, \nel \nPresidente\n \nde la República Ing. \nMartín Vizcarra,\n \ndispuso declarar \nde interés nacional y prioridad del Estado \nla lucha contra la violencia hacia las \nmuj\neres\n. En ese sentido\n, \nmediante la Resolución Suprema N° 129\n-\n2018\n-\nPCM, de \nfecha 08 de junio de 2018, se \ndispuso la creación de la Comisión Multisectorial de \n\nacciones para la protec\nción, prevención y atención de casos de violencia contra la \n\n. \n \n \n\nprevenir la violencia contra las mujeres, así como brindar protección y atención a las \nvíctimas de violencia\n\naprobado mediante Decreto Supremo N° 008\n-\n2018\n-\nMIMP y que contie

## NLP PIPELINE

In [65]:
def preprocess(text):
    for i in trange(len(text)):
        result= []
        for i in range(0,len(text)):
            text[i]=text[i].replace('y',' ')
            text[i]=text[i].replace('\x93',' ')
            text[i]=text[i].replace('\x94',' ')
            text[i]=text[i].replace('\n','')
            text[i]=text[i].replace('\g','')
            text[i]=text[i].replace('  ',' ')
            t=nlp(text[i].lower())
            [token.text for token in t if not token.is_punct and token.is_alpha]
            result.append(' '.join(list(t1)))
    return result

In [66]:
DOCU=preprocess(Caps)

100%|██████████| 26/26 [00:35<00:00,  1.37s/it]


In [50]:
#texto por página en el documento
DOCU

['separata especialministerio mujer y poblaciones vulnerablesdecreto supremo aprueba ﬁplan nacional violencia supremo nº julio consolidación mar historia contar',
 'martes julio legalesdecreto supremonº aprueba ﬁplan nacionalcontra violencia presidente repúblicaconsiderando artículo constitución política perú establece defensa persona humana respeto dignidad þ n supremo sociedad artículo nº orgánica ejecutivo establece competencia exclusiva ejecutivo diseñar supervisar políticas nacionales sectoriales cumplimiento obligatorio entidades niveles artículo decreto supremo nº establece constitu políticas nacionales obligatorio cumplimiento entidades gobierno nacional materia igualdad hombres mujeres referidas a impulsar sociedad acciones comunicaciones adopción valores prácticas actitudes comportamientos equitativos hombres mujeres garantizar derecho a discriminación mujeres erradicación violencia familiar decreto supremo nº aprueba ﬁplan nacional violencia mujer instrumento política públic

## Objeto doc spacy

In [67]:
doc_=[]
for i in range(0, len(DOCU)):
    doc=nlp(DOCU[i])
    doc_.append(doc)

In [52]:
doc_[9][3].dep_

'flat'

## Terminología y entidades

In [68]:
terms=[]
#terminologia, para hacer el glosario
for j in range(0,len(doc_)):
    for i in range(0,len(doc_[j])):
        if doc_[j][i].dep_ == 'amod':
            b=doc_[j][i-1].text,doc_[j][i].text
            for k in range(0,len(b)-1):
                c=b[k]+' '+b[k+1]
            terms.append(c)

In [54]:
terms

['separata especialministerio',
 'poblaciones vulnerablesdecreto',
 'nº julio',
 'julio legalesdecreto',
 'legalesdecreto supremonº',
 'violencia presidente',
 'artículo constitución',
 'constitución política',
 'defensa persona',
 'persona humana',
 'sociedad artículo',
 'orgánica ejecutivo',
 'competencia exclusiva',
 'exclusiva ejecutivo',
 'políticas nacionales',
 'nacionales sectoriales',
 'cumplimiento obligatorio',
 'obligatorio entidades',
 'entidades niveles',
 'constitu políticas',
 'políticas nacionales',
 'nacionales obligatorio',
 'gobierno nacional',
 'igualdad hombres',
 'hombres mujeres',
 'mujeres referidas',
 'acciones comunicaciones',
 'comunicaciones adopción',
 'adopción valores',
 'valores prácticas',
 'prácticas actitudes',
 'actitudes comportamientos',
 'comportamientos equitativos',
 'hombres mujeres',
 'mujeres erradicación',
 'violencia familiar',
 'mujer instrumento',
 'instrumento política',
 'política pública',
 'acción conjunta',
 'sociedad vigencia',
 'a

In [69]:
terms_cond=[]
#terminologia, para hacer el glosarioa=[]
for j in range(0,len(doc_)):
    for i in range(0,len(doc_[j])):
        if doc_[j][i].dep_ == 'compound' or doc_[j][i].dep_ =='nummod':
            b=doc_[j][i].text,doc_[j][i+1].text
            for k in range(0,len(b)-1):
                c=b[k]+' '+b[k+1]
            terms_cond.append(c)

In [70]:
ents=[]
for i in range(0, len(doc_)):
    for word in doc_[i].ents:
        if word.label_=='ORG' and 'MISC' and 'LOC':
            s=word.text
            ents.append(s)

In [71]:
#terminology_ME=list(set(terms+terms_cond+ents))
#terminology_PC=list(set(terms+terms_cond+ents))
terminology_PN=list(set(terms+terms_cond+ents))

In [39]:
#EJEMPLO DE DEPENDENCIAS
displacy.render(doc_[9], style='dep')

In [73]:
resMPC = list(set(terminology_ME) & set(terminology_PC))
resMPN = list(set(terminology_ME) & set(terminology_PN))
resPNPC = list(set(terminology_PN) & set(terminology_PC))

In [93]:
from tabulate import tabulate
Tabla = [['Documentos','% terminos iguales','Total de terminos primer documento'],
         ['Plan Nacional-Modelo Ecológico', (len(resMPN)/len(terminology_PN))*100,len(terminology_PN)],
         ['Modelo Ecológico-Plan Nacional', (len(resMPN)/len(terminology_ME))*100,len(terminology_ME)],
         ['Modelo Ecológico-Plan de acción', (len(resMPC)/len(terminology_PC))*100,len(terminology_PC)],
         ['Plan de acción-Modelo Ecológico', (len(resMPC)/len(terminology_ME))*100,len(terminology_ME)],
         ['Plan de acción-Plan Nacional', (len(resPNPC)/len(terminology_PN))*100,len(terminology_PN)],
         ['Plan Nacional-Plan de acción', (len(resPNPC)/len(terminology_PC))*100,len(terminology_PC)]]
         
        
print(tabulate(Tabla))

-------------------------------  ------------------  ----------------------------------
Documentos                       % terminos iguales  Total de terminos primer documento
Plan Nacional-Modelo Ecológico   5.3418803418803416  936
Modelo Ecológico-Plan Nacional   1.366867140513942   3658
Modelo Ecológico-Plan de acción  4.298874104401229   2931
Plan de acción-Modelo Ecológico  3.444505194095134   3658
Plan de acción-Plan Nacional     13.034188034188036  936
Plan Nacional-Plan de acción     4.162401910610713   2931
-------------------------------  ------------------  ----------------------------------
